## 1. Pre-training the Auto-Encoder

In this notebook, we pre-train the auto-encoder for reconstruction. In their paper, Carr et al. define the motion encoder `E_M` and the privacy encoder `E_P`, which are implemented identically as described below.

![Encoder Architecture](figures/encoder.png)

Here, the input size 75 refers to the frame length `T` of each sequence of 3D coordinates in the dataset. The output of each encoder is a motion embedding and a privacy embedding respectively. The acronyms are defined using the Pytorch implementations:

- C2D: Convolution 2D
- LR: Leaky ReLU
- Up: Upsample
- MP: Max Pooling
- RP2D: Reflection Pad 2D

This stage includes 5 epochs of paired pre-training, followed by 20 epochs of unpaired pre-training.

### 1.1 Paired Pre-training

In order to separate the embeddings, we first start by pre-training the auto-encoder in the paired setting for 5 epochs.

#### 1.1.1 Paired Data Loading

In the paired setting, the model is trained with "carefully matched sets of skeleton motions". We load the preprocessed paired samples of the NTU RGB+D dataset, where two distinct actors perform the same two actions under an identical camera view.

In [ ]:
from sitc.models import AE

ae = AE()